In [7]:
#@title Start up
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

%pip install ffmpeg

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

import numpy as np
import pandas as pd
import pickle
import cv2
import math
from google.colab.patches import cv2_imshow
from IPython.display import HTML

from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt

rootdir = "/content/drive/MyDrive/XAMI-MIMIC"
%matplotlib inline


# show all rows from dataframe
pd.set_option('display.max_rows', 100)
# show all columns from dataframe
pd.set_option('display.max_columns', 100)

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [112]:
#@title Init Variables
SCREEN_RES = [3180, 2160]

FONT_SIZE = 4

to_save_images = "/content/drive/MyDrive/REFLACX-plots/Time Study"
ignore_labels = ['xmin', 'ymin', 'xmax', 'ymax', 'certainty']

ellipses_labels = ['Acute fracture', 'Other', 'Pleural abnormality', 'Atelectasis', 'Pulmonary edema', 'Consolidation', 'Enlarged cardiac silhouette', 'Pneumothorax', 'Enlarged hilum', 'Interstitial lung disease', 'Groundglass opacity', 'Abnormal mediastinal contour', 'Lung nodule or mass', 'Pleural effusion', 'Fracture', 'Pleural thickening', 'High lung volume / emphysema', 'Hiatal hernia', 'Airway wall thickening', 'Emphysema', 'Nodule', 'Wide mediastinum', 'Fibrosis', 'Mass', 'Nothing']

with open("/content/drive/MyDrive/REFLACX-Study/Pickle/reflacx_dirs.pkl",'rb') as file:
  reflacx = pickle.load(file)

with open("/content/drive/MyDrive/REFLACX-Study/Pickle/timestudy_vars.pkl",'rb') as file:
  aux = pickle.load(file)
  certainty_labels = aux[0]
  fixations_in_ellipses = aux[1]

NORMAL_FIXS = 0
SILENCE_FIXS = 1
MS_FIXS = 2
#id in both match to correspond each other
with open("/content/drive/MyDrive/REFLACX-Study/Pickle/fixations.pkl",'rb') as file:
  #reflacx_fixations {id: [normal_fixs, silence_fixs, ms_fixs], ...}
  #fixations is array [x,y,raw_time,timestamp]
  reflacx_fixations = pickle.load(file)
with open("/content/drive/MyDrive/REFLACX-Study/Pickle/ellipses.pkl",'rb') as file:
  #reflacx_ellipses {id: ellipse_id:[[certainty,xmin,ymin,xmax,ymax,label]], ...}
  reflacx_ellipses = pickle.load(file)
X = 0
Y = 1 
CERTAINTY = 0
XMIN = 1
YMIN = 2
XMAX = 3
YMAX = 4
LABEL = 5

In [22]:
#@title Certainty by Ellipse Type
fixations_in_ellipses = {}

certainty_labels = {
    "Acute fracture": {"TIM": {}, "FIX": {}},
    "Other": {"TIM": {}, "FIX": {}},
    "Pleural abnormality": {"TIM": {}, "FIX": {}},
    "Atelectasis": {"TIM": {}, "FIX": {}},
    "Pulmonary edema": {"TIM": {}, "FIX": {}},
    "Consolidation": {"TIM": {}, "FIX": {}},
    "Enlarged cardiac silhouette": {"TIM": {}, "FIX": {}},
    "Pneumothorax": {"TIM": {}, "FIX": {}},
    "Enlarged hilum": {"TIM": {}, "FIX": {}},
    "Interstitial lung disease": {"TIM": {}, "FIX": {}},
    "Groundglass opacity": {"TIM": {}, "FIX": {}},
    "Abnormal mediastinal contour": {"TIM": {}, "FIX": {}},
    "Lung nodule or mass": {"TIM": {}, "FIX": {}},
    "Pleural effusion": {"TIM": {}, "FIX": {}},
    "Fracture": {"TIM": {}, "FIX": {}},
    "Pleural thickening": {"TIM": {}, "FIX": {}},
    "High lung volume / emphysema": {"TIM": {}, "FIX": {}},
    "Hiatal hernia": {"TIM": {}, "FIX": {}},
    "Airway wall thickening": {"TIM": {}, "FIX": {}},
    "Wide mediastinum": {"TIM": {}, "FIX": {}},
    "Fibrosis": {"TIM": {}, "FIX": {}},
}

def handleEllipses(label, certainties, timestamp, fixations):
    certainty = certainties[0] / certainties[1]
    if label == 'Mass' or label == 'Nodule':
      label = 'Lung nodule or mass'
    elif label == 'Emphysema':
      label = 'High lung volume / emphysema'
    
    certainty_labels[label]["TIM"].setdefault(timestamp, [0, 0])
    certainty_labels[label]["TIM"][timestamp][0] += certainty
    certainty_labels[label]["TIM"][timestamp][1] += 1
    certainty_labels[label]["FIX"].setdefault(fixations, [0, 0])
    certainty_labels[label]["FIX"][fixations][0] += certainty
    certainty_labels[label]["FIX"][fixations][1] += 1

In [23]:
#@title Certainty by Ellipse Type
fixations_in_ellipses1 = {}

certainty_labels1 = {
    "Acute fracture": {"TIM": {}, "FIX": {}},
    "Other": {"TIM": {}, "FIX": {}},
    "Pleural abnormality": {"TIM": {}, "FIX": {}},
    "Atelectasis": {"TIM": {}, "FIX": {}},
    "Pulmonary edema": {"TIM": {}, "FIX": {}},
    "Consolidation": {"TIM": {}, "FIX": {}},
    "Enlarged cardiac silhouette": {"TIM": {}, "FIX": {}},
    "Pneumothorax": {"TIM": {}, "FIX": {}},
    "Enlarged hilum": {"TIM": {}, "FIX": {}},
    "Interstitial lung disease": {"TIM": {}, "FIX": {}},
    "Groundglass opacity": {"TIM": {}, "FIX": {}},
    "Abnormal mediastinal contour": {"TIM": {}, "FIX": {}},
    "Lung nodule or mass": {"TIM": {}, "FIX": {}},
    "Pleural effusion": {"TIM": {}, "FIX": {}},
    "Fracture": {"TIM": {}, "FIX": {}},
    "Pleural thickening": {"TIM": {}, "FIX": {}},
    "High lung volume / emphysema": {"TIM": {}, "FIX": {}},
    "Hiatal hernia": {"TIM": {}, "FIX": {}},
    "Airway wall thickening": {"TIM": {}, "FIX": {}},
    "Wide mediastinum": {"TIM": {}, "FIX": {}},
    "Fibrosis": {"TIM": {}, "FIX": {}},
}

def handleEllipses1(label, certainties, timestamp, fixations):
    certainty = certainties[0] / certainties[1]
    if label == 'Mass' or label == 'Nodule':
      label = 'Lung nodule or mass'
    elif label == 'Emphysema':
      label = 'High lung volume / emphysema'
    
    certainty_labels1[label]["TIM"].setdefault(timestamp, [0, 0])
    certainty_labels1[label]["TIM"][timestamp][0] += certainty
    certainty_labels1[label]["TIM"][timestamp][1] += 1
    certainty_labels1[label]["FIX"].setdefault(fixations, [0, 0])
    certainty_labels1[label]["FIX"][fixations][0] += certainty
    certainty_labels1[label]["FIX"][fixations][1] += 1

In [127]:
#@title Study the relation between time of the report and number of clusters

def graphRelation():
  mean_cert_ms = {}
  mean_cert_sil = {}
  for id in reflacx_fixations:
    ms_in_ellipse = 0
    silence_in_ellipse = 0
    silence_time = reflacx_fixations[id][SILENCE_FIXS][-1][-1] if len(reflacx_fixations[id][SILENCE_FIXS]) > 0 else 0
    ms_time = reflacx_fixations[id][MS_FIXS][-1][-1] - silence_time if len(reflacx_fixations[id][MS_FIXS]) > 0 else 0
    for key, ellipses in reflacx_ellipses[id].items():
      for ellipse in ellipses:
        if len(reflacx_fixations[id][MS_FIXS]) > 0:
          for fixation in reflacx_fixations[id][MS_FIXS]:
            if ellipse[XMIN]<fixation[X]<ellipse[XMAX] and ellipse[YMIN]<fixation[Y]<ellipse[YMAX]:
              ms_in_ellipse += 1
        if len(reflacx_fixations[id][SILENCE_FIXS]) > 0:
          for fixation in reflacx_fixations[id][SILENCE_FIXS]:
            if ellipse[XMIN]<fixation[X]<ellipse[XMAX] and ellipse[YMIN]<fixation[Y]<ellipse[YMAX]:
              silence_in_ellipse += 1
        
        label = ellipse[LABEL]
        if label not in mean_cert_ms:
          mean_cert_ms[label] = [ellipse[CERTAINTY],1]
        else:
          mean_cert_ms[label][0] += ellipse[CERTAINTY]
          mean_cert_ms[label][1] += 1
        if label not in mean_cert_sil:
          mean_cert_sil[label] = [ellipse[CERTAINTY],1]
        else:
          mean_cert_sil[label][0] += ellipse[CERTAINTY]
          mean_cert_sil[label][1] += 1

    for key, value in mean_cert_sil.items():
      handleEllipses(key,value,silence_time,silence_in_ellipse)
    for key, value in mean_cert_ms.items():
      handleEllipses1(key,value,ms_time,ms_in_ellipse)
  

In [ ]:
#graphRelation()

In [132]:
#@title Draw Boxplots
#%%script echo skipping
for label in certainty_labels1:
  if label == 'High lung volume / emphysema':
    clabel = 'High lung volume_emphysema'
  else:
    clabel = label
    
  certainty_fix_graph = certainty_labels[label]['FIX']
  new_certainty_fix_graph = pd.DataFrame([[key, round(value[0] / value[1])] for key, value in certainty_fix_graph.items()])
  new_certainty_fix_graph.columns = ['Fixations','Certainty']
  
  fplot = sns.boxplot(data=new_certainty_fix_graph, x='Certainty', y='Fixations',palette='colorblind')
  fplot = sns.stripplot(data=new_certainty_fix_graph, x='Certainty', y='Fixations',jitter=True,alpha=0.5,color='black')
  fplot.figure.savefig(f"/content/drive/MyDrive/REFLACX-Study/Time Study/Certainty_MS/Fixations/{clabel}",format='jpeg',dpi=100)
  
  plt.clf()

  certainty_tim_graph = certainty_labels[label]['TIM']
  new_certainty_tim_graph = pd.DataFrame([[key, round(value[0] / value[1])] for key, value in certainty_tim_graph.items()])
  new_certainty_tim_graph.columns = ['Time','Certainty']
  
  tplot = sns.boxplot(data=new_certainty_tim_graph, x='Certainty', y='Time',palette='colorblind')
  tplot = sns.stripplot(data=new_certainty_tim_graph, x='Certainty', y='Time',jitter=True,alpha=0.5,color='black')
  tplot.figure.savefig(f"/content/drive/MyDrive/REFLACX-Study/Time Study/Certainty_MS/Time/{clabel}",format='jpeg',dpi=100)
  
  plt.clf()


<Figure size 640x480 with 0 Axes>

In [ ]:
#@title Only Ellipse Fixations
lung = np.array([0,0,0])
lung_t = 0
emph = np.array([0,0,0])
emph_t = 0
for item in fixations_in_ellipses:
  aitem = fixations_in_ellipses[item]
  aitem = np.array(aitem)
  sum = aitem.sum(axis=0)

  if item == 'Lung nodule or mass' or item == 'Mass' or item == 'Nodule':
    lung = np.add(lung,sum)
    lung_t += len(aitem)
  elif item == 'High lung volume / emphysema' or item == 'Emphysema':
    emph = np.add(lung,sum)
    emph_t += len(aitem)
  else:
    print(item, ',', sum/len(aitem))

print('Lung nodule or mass',',',lung/lung_t)
print('High lung volume / emphysema',',',emph/emph_t)

Acute fracture , [39.52173913 30.82608696  8.69565217]
Other , [71.68 45.28 26.4 ]
Pleural abnormality , [30.65463918 23.40378007  7.25085911]
Atelectasis , [43.24916388 31.49832776 11.75083612]
Pulmonary edema , [35.44895288 25.28926702 10.15968586]
Consolidation , [46.42980936 34.17244367 12.25736568]
Enlarged cardiac silhouette , [77.51126761 52.90985915 24.60140845]
Pneumothorax , [58.04132231 40.55371901 17.48760331]
Enlarged hilum , [38.84285714 25.94285714 12.9       ]
Interstitial lung disease , [45.13846154 29.53846154 15.6       ]
Groundglass opacity , [35.17567568 25.39189189  9.78378378]
Abnormal mediastinal contour , [69.08333333 47.36904762 21.71428571]
Pleural effusion , [35.58035714 26.92857143  8.65178571]
Fracture , [19.75   16.1875  3.5625]
Pleural thickening , [55.85714286 39.42857143 16.42857143]
Hiatal hernia , [80.04166667 46.33333333 33.70833333]
Airway wall thickening , [37.05555556 23.58333333 13.47222222]
Wide mediastinum , [80.3 51.  29.3]
Fibrosis , [39.1 2